In [5]:
import pandas as pd
from google.colab import files
files=files.upload()
df = pd.read_json('news_dataset.json')
df

Saving news_dataset.json to news_dataset.json


,text,category
0,Watching Schrödinger's Cat Die University of C...,SCIENCE
1,WATCH: Freaky Vortex Opens Up In Flooded Lake,SCIENCE
2,Entrepreneurs Today Don't Need a Big Budget to...,BUSINESS
3,These Roads Could Recharge Your Electric Car A...,BUSINESS
4,Civilian 'Guard' Fires Gun While 'Protecting' ...,CRIME
...,...,...
12690,Coach Shakes Hands Of Imaginary Players After ...,SPORTS
12691,This Minivan-Sized Sea Sponge Is Thought To Be...,SCIENCE
12692,RECAP: Dramatic Eclipse Photos Don't miss the ...,SCIENCE
12693,Richard Sherman Wants To Talk About Police Sho...,SPORTS


In [6]:
df.category.value_counts()

,count
category,
BUSINESS,4254
SPORTS,4167
CRIME,2893
SCIENCE,1381


In [7]:
coloumns_no = 1381
df_business = df[df.category == 'BUSINESS'].sample(coloumns_no, random_state=2022)
df_sports = df[df.category == 'SPORTS'].sample(coloumns_no, random_state=2022)
df_crime = df[df.category == 'CRIME'].sample(coloumns_no, random_state=2022)
df_science = df[df.category == 'SCIENCE'].sample(coloumns_no, random_state=2022)

In [10]:
df = pd.concat([df_business, df_crime, df_science, df_sports], axis = 0)
df.category.value_counts()

,count
category,
BUSINESS,1381
CRIME,1381
SCIENCE,1381
SPORTS,1381


In [11]:
df['category_num'] = df['category'].map({'BUSINESS':0, 'CRIME':1, 'SCIENCE':2, 'SPORTS':3})
df

,text,category,category_num
11967,GCC Business Leaders Remain Confident in the F...,BUSINESS,0
2912,From the Other Side; an Honest Review from Emp...,BUSINESS,0
3408,"Mike McDerment, CEO of FreshBooks, Talks About...",BUSINESS,0
502,How to Market Your Business While Traveling th...,BUSINESS,0
5279,How to Leverage Intuition in Decision-making I...,BUSINESS,0
...,...,...,...
4052,The Problem With Asking Football Players To Ac...,SPORTS,3
2292,NFL Clubs Seek to Feed at the Public Trough On...,SPORTS,3
10994,Boxer Wearing 'America 1st' Shorts Gets Pounde...,SPORTS,3
9359,Garbine Muguruza Blows Away Venus Williams To ...,SPORTS,3


In [13]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 3.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [14]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [15]:
df['text_vector'] = df['text'].apply(lambda text:nlp(text).vector)


In [16]:
df

,text,category,category_num,text_vector
11967,GCC Business Leaders Remain Confident in the F...,BUSINESS,0,"[-0.056873698, 0.14779751, -0.03915794, -0.064..."
2912,From the Other Side; an Honest Review from Emp...,BUSINESS,0,"[-0.009033982, 0.13833478, -0.1900449, -0.0009..."
3408,"Mike McDerment, CEO of FreshBooks, Talks About...",BUSINESS,0,"[-0.07396942, 0.23576386, -0.15229684, -0.0325..."
502,How to Market Your Business While Traveling th...,BUSINESS,0,"[0.01922901, 0.17811063, -0.10048069, -0.13636..."
5279,How to Leverage Intuition in Decision-making I...,BUSINESS,0,"[-0.044400495, 0.23665424, -0.13312624, 0.0050..."
...,...,...,...,...
4052,The Problem With Asking Football Players To Ac...,SPORTS,3,"[-0.071693085, 0.027155658, -0.2631884, -0.144..."
2292,NFL Clubs Seek to Feed at the Public Trough On...,SPORTS,3,"[0.10028562, 0.15855752, -0.060141183, -0.0777..."
10994,Boxer Wearing 'America 1st' Shorts Gets Pounde...,SPORTS,3,"[-0.067053765, 0.017008958, -0.050246447, 0.02..."
9359,Garbine Muguruza Blows Away Venus Williams To ...,SPORTS,3,"[0.062104926, 0.18616068, 0.08357925, 0.100631..."


In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.text_vector, df.category_num, test_size=0.3, random_state=2022)

In [18]:
import numpy as np

X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)

In [22]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()
scaled_train_embed = scaler.fit_transform(X_train_2d)
scaled_test_embed = scaler.transform(X_test_2d)

In [24]:
clf = MultinomialNB()
clf.fit(scaled_train_embed, y_train)

MultinomialNB()

In [26]:
from sklearn.metrics import classification_report

y_pred = clf.predict(scaled_test_embed)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.86      0.79       411
           1       0.87      0.87      0.87       416
           2       0.84      0.84      0.84       387
           3       0.87      0.74      0.80       444

    accuracy                           0.82      1658
   macro avg       0.83      0.83      0.83      1658
weighted avg       0.83      0.82      0.82      1658



In [31]:
#with preprocessed data

def preprocessed(text):
  doc = nlp(text)
  new_val = []

  for token in doc:
    if token.is_stop or token.is_punct:
      continue
    else:
      new_val.append(token.lemma_)

  return ' '.join(new_val)

In [32]:
df['preprocessed_text'] = df['text'].apply(preprocessed)
df

,text,category,category_num,text_vector,preprocessed_text
11967,GCC Business Leaders Remain Confident in the F...,BUSINESS,0,"[-0.056873698, 0.14779751, -0.03915794, -0.064...",GCC Business leader remain confident face regi...
2912,From the Other Side; an Honest Review from Emp...,BUSINESS,0,"[-0.009033982, 0.13833478, -0.1900449, -0.0009...",Honest Review employee wake morning love impor...
3408,"Mike McDerment, CEO of FreshBooks, Talks About...",BUSINESS,0,"[-0.07396942, 0.23576386, -0.15229684, -0.0325...",Mike McDerment ceo FreshBooks talk give build ...
502,How to Market Your Business While Traveling th...,BUSINESS,0,"[0.01922901, 0.17811063, -0.10048069, -0.13636...",market business travel World recently amazing ...
5279,How to Leverage Intuition in Decision-making I...,BUSINESS,0,"[-0.044400495, 0.23665424, -0.13312624, 0.0050...",leverage intuition decision making feel safe r...
...,...,...,...,...,...
4052,The Problem With Asking Football Players To Ac...,SPORTS,3,"[-0.071693085, 0.027155658, -0.2631884, -0.144...",Problem ask Football Players act like White Gu...
2292,NFL Clubs Seek to Feed at the Public Trough On...,SPORTS,3,"[0.10028562, 0.15855752, -0.060141183, -0.0777...",NFL Clubs seek feed Public Trough National Foo...
10994,Boxer Wearing 'America 1st' Shorts Gets Pounde...,SPORTS,3,"[-0.067053765, 0.017008958, -0.050246447, 0.02...",Boxer wear America 1st Shorts get pound Mexica...
9359,Garbine Muguruza Blows Away Venus Williams To ...,SPORTS,3,"[0.062104926, 0.18616068, 0.08357925, 0.100631...",Garbine Muguruza blow away Venus Williams Wimb...


In [34]:
df['vector'] = df['text'].apply(lambda text: nlp(text).vector)
df

,text,category,category_num,text_vector,preprocessed_text,vector
11967,GCC Business Leaders Remain Confident in the F...,BUSINESS,0,"[-0.056873698, 0.14779751, -0.03915794, -0.064...",GCC Business leader remain confident face regi...,"[-0.056873698, 0.14779751, -0.03915794, -0.064..."
2912,From the Other Side; an Honest Review from Emp...,BUSINESS,0,"[-0.009033982, 0.13833478, -0.1900449, -0.0009...",Honest Review employee wake morning love impor...,"[-0.009033982, 0.13833478, -0.1900449, -0.0009..."
3408,"Mike McDerment, CEO of FreshBooks, Talks About...",BUSINESS,0,"[-0.07396942, 0.23576386, -0.15229684, -0.0325...",Mike McDerment ceo FreshBooks talk give build ...,"[-0.07396942, 0.23576386, -0.15229684, -0.0325..."
502,How to Market Your Business While Traveling th...,BUSINESS,0,"[0.01922901, 0.17811063, -0.10048069, -0.13636...",market business travel World recently amazing ...,"[0.01922901, 0.17811063, -0.10048069, -0.13636..."
5279,How to Leverage Intuition in Decision-making I...,BUSINESS,0,"[-0.044400495, 0.23665424, -0.13312624, 0.0050...",leverage intuition decision making feel safe r...,"[-0.044400495, 0.23665424, -0.13312624, 0.0050..."
...,...,...,...,...,...,...
4052,The Problem With Asking Football Players To Ac...,SPORTS,3,"[-0.071693085, 0.027155658, -0.2631884, -0.144...",Problem ask Football Players act like White Gu...,"[-0.071693085, 0.027155658, -0.2631884, -0.144..."
2292,NFL Clubs Seek to Feed at the Public Trough On...,SPORTS,3,"[0.10028562, 0.15855752, -0.060141183, -0.0777...",NFL Clubs seek feed Public Trough National Foo...,"[0.10028562, 0.15855752, -0.060141183, -0.0777..."
10994,Boxer Wearing 'America 1st' Shorts Gets Pounde...,SPORTS,3,"[-0.067053765, 0.017008958, -0.050246447, 0.02...",Boxer wear America 1st Shorts get pound Mexica...,"[-0.067053765, 0.017008958, -0.050246447, 0.02..."
9359,Garbine Muguruza Blows Away Venus Williams To ...,SPORTS,3,"[0.062104926, 0.18616068, 0.08357925, 0.100631...",Garbine Muguruza blow away Venus Williams Wimb...,"[0.062104926, 0.18616068, 0.08357925, 0.100631..."


In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.vector, df.category_num, test_size=0.3, random_state=2022)

In [36]:
import numpy as np

X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)

In [37]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()
scaled_train_embed = scaler.fit_transform(X_train_2d)
scaled_test_embed = scaler.transform(X_test_2d)

In [38]:
clf = MultinomialNB()
clf.fit(scaled_train_embed, y_train)

MultinomialNB()

In [39]:
from sklearn.metrics import classification_report

y_pred = clf.predict(scaled_test_embed)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.86      0.79       411
           1       0.87      0.87      0.87       416
           2       0.84      0.84      0.84       387
           3       0.87      0.74      0.80       444

    accuracy                           0.82      1658
   macro avg       0.83      0.83      0.83      1658
weighted avg       0.83      0.82      0.82      1658

